<a href="https://colab.research.google.com/github/otzhora/python_learning/blob/master/Do_deep_conv_nets_needs_to_be.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf
import os

import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [0]:
batch_size = 64
num_classes = 10
epochs = 100
data_augumentations = True
num_predictions = 20

In [0]:
def get_model(x_train):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

  return model

In [0]:
def get_data():
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  return (x_train, y_train), (x_test, y_test)

In [0]:
def train_model():
  (x_train, y_train), (x_test, y_test) = get_data()
  model = get_model(x_train)
  datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.2)
  datagen.fit(x_train)

  # Fit the model on the batches generated by datagen.flow().
  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=1)
  return model

In [26]:
model = train_model()

Epoch 1/100
782/782 [==============================] - 89s 114ms/step - loss: 1.9447 - accuracy: 0.2848 - val_loss: 1.6642 - val_accuracy: 0.4084
Epoch 2/100
782/782 [==============================] - 91s 116ms/step - loss: 1.6705 - accuracy: 0.3898 - val_loss: 1.4956 - val_accuracy: 0.4555
Epoch 3/100
782/782 [==============================] - 91s 116ms/step - loss: 1.5446 - accuracy: 0.4380 - val_loss: 1.4748 - val_accuracy: 0.4699
Epoch 4/100
782/782 [==============================] - 92s 117ms/step - loss: 1.4710 - accuracy: 0.4668 - val_loss: 1.3089 - val_accuracy: 0.5287
Epoch 5/100
782/782 [==============================] - 91s 116ms/step - loss: 1.4076 - accuracy: 0.4938 - val_loss: 1.2785 - val_accuracy: 0.5370
Epoch 6/100
782/782 [==============================] - 90s 116ms/step - loss: 1.3493 - accuracy: 0.5176 - val_loss: 1.2028 - val_accuracy: 0.5737
Epoch 7/100
782/782 [==============================] - 91s 116ms/step - loss: 1.3055 - accuracy: 0.5334 - val_loss: 1.1964 -

In [0]:
(_, _), (x_test, y_test) = get_data()

In [29]:
scores = model.evaluate(x_test, y_test, verbose=1)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [30]:
scores

[0.6349036903381348, 0.7804]